## Import Libraries needed

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  48.57 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  29.97 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  36.89 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  40.44 MB/s
Libraries imported.


## Download data from the web and convert to a Pandas Dataframe using BeautifulSoup

In [2]:
from bs4 import BeautifulSoup

data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(data.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


### Make the 1st row headers

In [3]:
headers = df.loc[0]
df = df[1:]
df.columns = headers
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### Drop rows with Borough not assigned

In [4]:
df[df['Borough'] != 'Not assigned'].count()

0
Postcode         212
Borough          212
Neighbourhood    212
dtype: int64

In [5]:
df = df[df['Borough'] != 'Not assigned']
df.shape

(212, 3)

### Merge Duplicate rows with same Borough into one

In [6]:
df = df.groupby('Postcode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join}).reset_index()
df.head()

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


### Replace Not assigned neighborhood with Borough Name

In [7]:
df[df['Neighbourhood'] == 'Not assigned']

,Postcode,Neighbourhood,Borough
85,M7A,Not assigned,Queen's Park


In [8]:
df['Neighbourhood'] = df['Neighbourhood'].apply(lambda x: df.Borough if x == 'Not assigned' else df.Neighbourhood)
df[df['Neighbourhood'] == 'Not assigned']

,Postcode,Neighbourhood,Borough


In [9]:
df.shape

(103, 3)